In [1]:
# Use DeepSurv from the repo
import sys
sys.path.append('/home/zahrael97/deepsurv')
import deepsurv
sys.path.append('/home/zahrael97/anaconda3/lib/python3.7/site-packages/deepsurv/deepsurv_logger.py')
import deepsurv.deep_surv
from deepsurv.deepsurv_logger import DeepSurvLogger, TensorboardLogger
import utils
import viz

import numpy as np
import pandas as pd

import lasagne
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# Read in dataset
First, I read in the dataset and print the first five elements to get a sense of what the dataset looks like

In [2]:
train_path = '/home/zahrael97/Downloads/challenge file/xtrain/data_Q0G7b5t'

df_Radiomics_train = pd.read_csv(train_path + '/features/radiomics.csv',skiprows=[0,2], header=[0],index_col=0)

df_clinical_train = pd.read_csv(train_path + '/features/clinical_data.csv',index_col=0)
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
df_clinical_train = df_clinical_train.fillna(df_clinical_train.mean())
encoder = LabelEncoder()
encoder.fit(df_clinical_train["SourceDataset"] )
df_clinical_train["SourceDataset"]= encoder.transform(df_clinical_train["SourceDataset"])

In [3]:
df_X_train = pd.concat([df_Radiomics_train[["original_shape_Sphericity", "original_shape_SurfaceVolumeRatio", "original_shape_Maximum3DDiameter","original_firstorder_Entropy",
"original_glcm_Id","original_glcm_Idm"]],df_clinical_train[["SourceDataset","Nstage","age"]]], axis=1)

In [4]:
ytrain_path = '/home/zahrael97/Downloads/challenge file/ytrain'
df_y = pd.read_csv( ytrain_path+ '/output_VSVxRFU.csv', index_col=0)
p0 = df_y.loc[202]
print("p0.Event", p0.Event)
print("p0.SurvivalTime", p0.SurvivalTime)

p0.Event 0
p0.SurvivalTime 1378


In [5]:
#from sklearn.model_selection import train_test_split

#y_train, y_test = train_test_split(df_y, test_size = 0.2, random_state=42)

In [6]:
df_train = pd.concat([df_X_train, df_y], axis = 1)

# Transform the dataset to "DeepSurv" format
DeepSurv expects a dataset to be in the form:

    {
        'x': numpy array of float32
        'e': numpy array of int32
        't': numpy array of float32
        'hr': (optional) numpy array of float32
    }
    
You are providing me a csv, which I read in as a pandas dataframe. Then I convert the pandas dataframe into the DeepSurv dataset format above. 

In [7]:
df_train

,original_shape_Sphericity,original_shape_SurfaceVolumeRatio,original_shape_Maximum3DDiameter,original_firstorder_Entropy,original_glcm_Id,original_glcm_Idm,SourceDataset,Nstage,age,SurvivalTime,Event
202,0.650210,0.275228,48.559242,5.138062,0.338672,0.262780,1,0,66.000000,1378,0
371,0.573079,0.240727,75.703368,4.461054,0.495719,0.439879,0,2,64.572200,379,1
246,0.642913,0.200766,70.434367,3.437111,0.616607,0.581458,0,3,66.045200,573,1
240,0.634469,0.323878,46.818800,4.352564,0.457545,0.391681,0,2,59.356600,959,0
284,0.584242,0.327241,53.795911,4.055700,0.544607,0.501892,0,3,71.055400,2119,0
...,...,...,...,...,...,...,...,...,...,...,...
261,0.768658,0.281441,36.619667,4.747964,0.451176,0.383987,0,0,87.126600,1540,0
298,0.366382,0.332614,122.531629,2.716180,0.684890,0.666367,0,0,68.766856,946,0
129,0.616308,0.224665,64.876806,2.368997,0.746369,0.735862,0,1,59.843900,559,0
273,0.721570,0.605798,22.472205,5.129264,0.272701,0.192854,1,0,70.000000,1952,0


In [8]:
# event_col is the header in the df that represents the 'Event / Status' indicator
# time_col is the header in the df that represents the event time
def dataframe_to_deepsurv_ds(df_train, event_col = 'Event', time_col = 'SurvivalTime'):
    # Extract the event and time columns as numpy arrays
    e = df_train[event_col].values.astype(np.int32)
    t = df_train[time_col].values.astype(np.float32)

    # Extract the patient's covariates as a numpy array
    x_df = df_train.drop([event_col, time_col], axis = 1)
    x = x_df.values.astype(np.float32)
    
    # Return the deep surv dataframe
    return {
        'x' : x,
        'e' : e,
        't' : t
    }

# If the headers of the csv change, you can replace the values of 
# 'event_col' and 'time_col' with the names of the new headers
# You can also use this function on your training dataset, validation dataset, and testing dataset
train_data = dataframe_to_deepsurv_ds(df_train, event_col = 'Event', time_col= 'SurvivalTime')

In [9]:
train_data

{'x': array([[ 0.65021026,  0.27522776, 48.559242  , ...,  1.        ,
          0.        , 66.        ],
        [ 0.57307863,  0.24072683, 75.70337   , ...,  0.        ,
          2.        , 64.5722    ],
        [ 0.64291304,  0.20076598, 70.434364  , ...,  0.        ,
          3.        , 66.0452    ],
        ...,
        [ 0.61630803,  0.22466502, 64.87681   , ...,  0.        ,
          1.        , 59.8439    ],
        [ 0.72157025,  0.6057985 , 22.472204  , ...,  1.        ,
          0.        , 70.        ],
        [ 0.7033614 ,  0.18380603, 67.54258   , ...,  0.        ,
          3.        , 54.694     ]], dtype=float32),
 'e': array([0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1,
        0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
        1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,

Now once you have your dataset all formatted, define you hyper_parameters as a Python dictionary. 
I'll provide you with some example hyper-parameters, but you should replace the values once you tune them to your specific dataset

In [10]:
hyperparams = {'L2_reg': 10.0,
    'batch_norm': True,
    'dropout': 0.4,
    'hidden_layers_sizes': [25, 25],
    'learning_rate': 1e-05,
    'lr_decay': 0.001,
    'momentum': 0.9,
    'n_in': train_data['x'].shape[1],
    'standardize': True}

Once you prepared your dataset, and defined your hyper-parameters. Now it's time to train DeepSurv!

In [11]:
# Create an instance of DeepSurv using the hyperparams defined above
model = deepsurv.DeepSurv(**hyperparams)
# DeepSurv can now leverage TensorBoard to monitor training and validation
# This section of code is optional. If you don't want to use the tensorboard logger
# Uncomment the below line, and comment out the other three lines: 
# logger = None

experiment_name = 'test_experiment_sebastian'
logdir = './logs/tensorboard/'
logger = TensorboardLogger(experiment_name, logdir=logdir)

# Now we train the model
update_fn=lasagne.updates.nesterov_momentum # The type of optimizer to use. \
                                            # Check out http://lasagne.readthedocs.io/en/latest/modules/updates.html \
                                            # for other optimizers to use
n_epochs = 2000

# If you have validation data, you can add it as the second parameter to the function
metrics = model.train(train_data, n_epochs=n_epochs, logger=logger, update_fn=update_fn)

2020-02-10 19:35:15,093 - Training step 0/2000    |                         | - loss: 19.4218 - ci: 0.4802
2020-02-10 19:35:44,946 - Training step 250/2000  |***                      | - loss: 10.2858 - ci: 0.5593
2020-02-10 19:36:13,675 - Training step 500/2000  |******                   | - loss: 6.9643 - ci: 0.5709
2020-02-10 19:36:40,628 - Training step 750/2000  |*********                | - loss: 5.7963 - ci: 0.5861
2020-02-10 19:37:13,349 - Training step 1000/2000 |************             | - loss: 5.3891 - ci: 0.6093
2020-02-10 19:37:45,904 - Training step 1250/2000 |***************          | - loss: 5.2604 - ci: 0.6260
2020-02-10 19:38:17,252 - Training step 1500/2000 |******************       | - loss: 5.1971 - ci: 0.6469
2020-02-10 19:38:46,720 - Training step 1750/2000 |*********************    | - loss: 5.1803 - ci: 0.6608
2020-02-10 19:39:18,530 - Finished Training with 2000 iterations in 243.54s


In [12]:
test_path = '/home/zahrael97/Downloads/challenge file/xtest/data_9Cbe5hx'
df_Radiomics_test = pd.read_csv(test_path +'/features/radiomics.csv',skiprows=[0,2], header=[0],index_col=0)
df_clinical_test = pd.read_csv(test_path + '/features/clinical_data.csv',index_col=0)

In [13]:
df_clinical_test = df_clinical_test.fillna(df_clinical_test.mean())
encoder = LabelEncoder()
encoder.fit(df_clinical_test["SourceDataset"])
df_clinical_test["SourceDataset"] = encoder.transform(df_clinical_test["SourceDataset"])

In [14]:
df_X_test = pd.concat([df_Radiomics_test[["original_shape_Sphericity", "original_shape_SurfaceVolumeRatio", "original_shape_Maximum3DDiameter","original_firstorder_Entropy",
"original_glcm_Id","original_glcm_Idm"]],df_clinical_test[["SourceDataset","Nstage","age"]]], axis=1)

In [15]:
# Print the final metrics
print('Train C-Index:', metrics['c-index'][-1])

Train C-Index: (1999, 0.6700196115644967)


In [17]:
#test_data = dataframe_to_deepsurv_ds(df_X_test, event_col = 'Event', time_col= 'SurvivalTime')

In [18]:
#metrics_test = model.train(test_data, n_epochs=n_epochs, logger=logger, update_fn=update_fn)

In [19]:
# Print the final metrics
#print('Train C-Index:', metrics_test['c-index'][-1])